## 0) Setup conda environment
Colab does not include conda by default. This cell installs Miniconda, creates the environment from `environment.yaml`, and activates it.

In [ ]:
# Install Miniconda (Colab)
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /content/miniconda.sh
!bash /content/miniconda.sh -b -p /content/miniconda

# Initialize conda in this notebook
import os
os.environ["PATH"] = "/content/miniconda/bin:" + os.environ["PATH"]

# Create and activate env
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r


# MyControlNet Colab Workflow
Follow the steps below to prepare data, add control weights, and train, based on the README.

## 1) Clone repo and set working directory
If the repo is already in your Drive, skip the clone and just `cd` into it.

In [ ]:
# Optional: mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Clone repo (or skip if already present in Drive)
!git clone https://github.com/robinrj6/MyControlNet.git /content/MyControlNet

# Set working directory
import os
os.chdir('/content/MyControlNet')
print('CWD:', os.getcwd())

# setup conda environment
!conda env create -f environment.yaml -n control
!conda run -n control python -V

## 2) Download and extract Fill50K dataset
Download the dataset and extract it into the `training/` folder so you have `training/fill50k/`.

In [ ]:
# Replace with the actual dataset URL
DATASET_URL = "https://huggingface.co/lllyasviel/ControlNet/resolve/main/training/fill50k.zip"
!wget -O /content/fill50k.zip "$DATASET_URL"
!mkdir -p /content/MyControlNet/training
!unzip -q /content/fill50k.zip -d /content/MyControlNet/training

# Check folder structure
!ls -la /content/MyControlNet/training
!ls -la /content/MyControlNet/training/fill50k

## 3) Verify dataset
Run the dataset test and confirm the printed shapes look correct.

In [ ]:
# Run dataset test
!python tutorial_dataset_test.py

## 4) Download model checkpoints
Place the .ckpt files into the `models/` folder and generate `control_sd15_ini.ckpt`.

In [ ]:
# Download or upload checkpoints into models/
# Example placeholders (replace with real URLs or upload manually):
# !mkdir -p models
!wget -O models/v1-5-pruned.ckpt "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt"
# !wget -O models/v2-1_512-ema-pruned.ckpt "https://huggingface.co/Manojb/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt"

# Create control checkpoint
!python tool_add_control.py ./models/v1-5-pruned.ckpt ./models/control_sd15_ini.ckpt

## 5) Train
Start training after dataset and checkpoints are ready.

In [ ]:
# Start training
# !python tutorial_train.py